# PLOT SEA LEVEL PRESSURE AND 1000-500 THICKNESS

In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import metpy.calc as mpcalc
from metpy.units import units
import sys
import time

from wrf import (getvar, interplevel, to_np, latlon_coords, get_cartopy,
                 cartopy_xlim, cartopy_ylim, extract_times, ALL_TIMES, interpz3d, g_geoht,vinterp)

In [2]:
def plot_maxmin_points(lon, lat, data, extrema, nsize, symbol, color='k',
                     plotValue=True, transform=None):
    """
    This function will find and plot relative maximum and minimum for a 2D grid. The function
    can be used to plot an H for maximum values (e.g., High pressure) and an L for minimum
    values (e.g., low pressue). It is best to used filetered data to obtain  a synoptic scale
    max/min value. The symbol text can be set to a string value and optionally the color of the
    symbol and any plotted value can be set with the parameter color
    lon = plotting longitude values (2D)
    lat = plotting latitude values (2D)
    data = 2D data that you wish to plot the max/min symbol placement
    extrema = Either a value of max for Maximum Values or min for Minimum Values
    nsize = Size of the grid box to filter the max and min values to plot a reasonable number
    symbol = String to be placed at location of max/min value
    color = String matplotlib colorname to plot the symbol (and numerica value, if plotted)
    plot_value = Boolean (True/False) of whether to plot the numeric value of max/min point
    The max/min symbol will be plotted on the current axes within the bounding frame
    (e.g., clip_on=True)
    """
    from scipy.ndimage.filters import maximum_filter, minimum_filter

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], symbol, color=color, size=24,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)
        ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]],
                '\n' + str(np.int(data[mxy[i], mxx[i]])),
                color=color, size=12, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform)

In [3]:
#Define type of tile and desired pressure level
#fullphys_expdom or nolatentheat_expdom
filetype = "nolatentheat_expdom"                                   #CHANGE

In [4]:
# Open the NetCDF file
ncfile = Dataset("wrfout_"+filetype+".nc", "r")

# Extract the pressure, geopotential height, and wind variables
pressure = getvar(ncfile, "pressure",timeidx=ALL_TIMES)
z = getvar(ncfile, "z", timeidx=ALL_TIMES,units="m")
thta = getvar(ncfile, "th",timeidx=ALL_TIMES)
tmpk = getvar(ncfile, "tk",timeidx=ALL_TIMES)
ua = getvar(ncfile, "ua", timeidx=ALL_TIMES, units="kt")
va = getvar(ncfile, "va", timeidx=ALL_TIMES, units="kt")
slp = getvar(ncfile, "slp", timeidx = ALL_TIMES, units="hPa")

In [5]:
#Define constants
R = 287.05 #J kg^-1 K^-1
kappa=0.2856219 #Rd / cp
cp = 1004.7 #J kg^-1 K^-1
pref = 1000. #hPa
g = 9.80665 #gravity
omega = 7.292e-5 #Earth's rotation rate (s^-1)
a = 2.e7 / np.pi #Radius of the earth (m)
p0=1.e5
alpha=-1./5.255877
gamma = 0.0065 #lapse rate, (K/m)

In [6]:
#Define pressure level array
pb = 1000. #base pressure
pb_half = 975.
pt = 100. #top pressure
pt_half = 125.
p = np.linspace(pb,pt,19) #make array of size 19 with even spacing from base pressure to top pressure
#p_half = np.linspace(pb_half,pt_half,18)
delp = (p[0] - p[1]) * 100. #Convert to Pa

In [7]:
#Manually define phalf based on length of k and pressure values
phalf = np.zeros((len(p)-1))
for i in range(len(p)-1):
    phalf[i] = (p[i+1] + p[i]) / 2.

In [8]:
#Bounds for inversion
lat1 = 30. #Southern boundary
lat2 = 70. #Northern boundary
lon1 = 150. #Western boundary
lon2 = 250. #Eastern boundary
dlat = lat2 - lat1
dlon = lon2 - lon1
dellatstar = (np.pi/180.) * a
dellonstar = (np.pi/180.) * a
sigma1 = dellonstar / dellatstar
sigma2 = dellonstar / delp

In [9]:
#Get times from dataset
times = extract_times(ncfile, None)
count = np.count_nonzero(times)
#Set t, k, i, j indice lengths
tlen = len(times)
klen = len(p)
ilen = np.shape(z[:,:,:,:])[2] #This is really the j index (north-south index)
jlen = np.shape(z[:,:,:,:])[3] #This is really the i index (east-west index)

In [10]:
hght = np.zeros((tlen, klen, ilen, jlen))
levels = [1000,950,900,850,800,750,700,650,600,550,500,450,400,350,300,250,200,150,100]
for t in range(tlen):
    hght[t,:,:,:] = vinterp(ncfile,field=z[t,:,:,:],vert_coord='p',interp_levels=levels,extrapolate=True, log_p=True, field_type='z',timeidx=t)

In [11]:
lat = getvar(ncfile, "lat")
lon = getvar(ncfile, "lon")

In [12]:
latmax = float(np.max(lat))
latmin = float(np.min(lat))
lonmax = float(np.max(lon))
lonmin = float(np.min(lon))

In [13]:
lat = np.arange(latmin,latmax,((latmax-latmin)/jlen))
lon = np.arange(lonmin,lonmax,((lonmax-lonmin)/ilen))
lenlat = len(lat)
lenlon = len(lon)

In [14]:
#Calculate Coriolis parameter
lenlat = len(lat)
cor = np.zeros((lenlat))
for j in range(lenlat):
    cor[j] = 2. * omega * np.sin(lat[j]*(np.pi / 180.)) # degrees to radians

In [15]:
#Calculate Rossby parameter (beta), variation in Coriolis force with respect to changing latitude
beta = np.zeros((lenlat))
for j in range(lenlat):
    beta[j] = (2. * omega / a) * np.cos(lat[j] * (np.pi / 180.))

In [16]:
#Calculate mean Coriolis Parameter weighted by latitude
fo = 0.
for j in range(lenlat):
    fo = fo + (cor[j] / (dlat+1)) #fo = fo + (cor[j] / (jlen-1))

In [17]:
#Format times to be in standard format
dattimes = []
import pandas as pd
for i in range (0, count):
    dattimes.append(str(pd.Timestamp(times[i])))

In [18]:
thick = hght[:,10,:,:] - hght[:,0,:,:] #10 is 500 hPa, 0 is 1000 hPa

In [20]:
lats, lons = latlon_coords(z)
dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

In [21]:
lats = np.asarray(lats)
lons = np.asarray(lons)

In [22]:
lats_redo = lats
lons_redo = lons

In [23]:
for i in range(ilen):
    for j in range(jlen):
        if lons_redo[i,j] < 0.0:
            lons_redo[i,j] = lons_redo[i,j] + 360.
        else:
            lons_redo[i,j] = lons_redo[i,j]

In [24]:
from scipy.ndimage.filters import gaussian_filter

In [26]:
#PLOTTING FOR sea level pressure
for i in range (0,count):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=ncfile)
    
    lats, lons = latlon_coords(z)
    dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)
    
    #Plot qgpvmean
    tmp1 = np.squeeze(slp[i,:,:])
    tmp2 = np.squeeze(thick[i,:,:])
    
    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons), to_np(lats), to_np(tmp1), vmin = 8750., vmax=10000.,
                                   #cmap = "bwr", transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    #FOR PROPER, PAPER-READY PLOTS
    levels = np.arange(960.,1060.,4)
    qgcs = plt.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp1),3),levels=levels, transform=crs.PlateCarree(), colors = "black")
    plt.clabel(qgcs, fmt='%.1f', fontsize=10.,inline=True)
    
    # Plot thickness with multiple colors
    clevs = (np.arange(0, 5400, 60),
             np.array([5400]),
             np.arange(5460, 7000, 60))
    colors = ('tab:blue', 'b', 'tab:red')
    kw_clabels = {'fontsize': 11, 'inline': True, 'inline_spacing': 5, 'fmt': '%i',
                  'rightside_up': True, 'use_clabeltext': True}
    for clevthick, color in zip(clevs, colors):
        cs = ax.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp2),3), levels=clevthick, colors=color,
                    linewidths=1.0, linestyles='dashed', transform=crs.PlateCarree())
        plt.clabel(cs, **kw_clabels)
        
    # Use definition to plot H/L symbols
    plot_maxmin_points(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp1),3), 'max', 50, symbol='H', color='b',  transform=crs.PlateCarree())
    plot_maxmin_points(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp1),3), 'min', 25, symbol='L', color='r', transform=crs.PlateCarree())
    
    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))
    
    #Zoom-in option
    #ax.set_extent([-150, -110, 40, 60]) #lon,lon,lat,lat

    ax.gridlines(color="black", linestyle="dotted")

    ax.set_title('WRF Sea Level Pressure (hPa) and 1000-500 hPa Thickness (m) \n Valid Time: {} UTC'.format(dattimes[i]))
    
    plt.savefig('plots/sfc/SFC_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

Working on time:  2019-11-25 12:00:00
Working on time:  2019-11-25 15:00:00
Working on time:  2019-11-25 18:00:00
Working on time:  2019-11-25 21:00:00
Working on time:  2019-11-26 00:00:00
Working on time:  2019-11-26 03:00:00
Working on time:  2019-11-26 06:00:00
Working on time:  2019-11-26 09:00:00
Working on time:  2019-11-26 12:00:00
Working on time:  2019-11-26 15:00:00
Working on time:  2019-11-26 18:00:00
Working on time:  2019-11-26 21:00:00
Working on time:  2019-11-27 00:00:00
Working on time:  2019-11-27 03:00:00
Working on time:  2019-11-27 06:00:00
Working on time:  2019-11-27 09:00:00
Working on time:  2019-11-27 12:00:00
Working on time:  2019-11-27 15:00:00
Working on time:  2019-11-27 18:00:00
Working on time:  2019-11-27 21:00:00
Working on time:  2019-11-28 00:00:00
